#  Traffic Prediction - Multi-Class Imbalanced Classification


In this notebook, I will try to build a model that predicts the traffic situation in New York City, according to 3 levels: 0-Low,1-Medium ,2-High.

Using this model it will be possible to assess the future traffic situation within the city, based on a small number of features.

Given the 3 data sets, I will choose to use "nyc_traffic_ml_orig".


### A brief explanation of the variables in the table:

- Segment_ID: is an identifier for each street segment.

- Is_Weekend: tells whether it's a weekend or not.

- Daylight: lighting percentage (the hours of the day placed in a bell-curving function f(x) = (2)/(1 + np.exp(((x - 12)/4)^2))).

- Season1-4: One-Hot-Encoded Seasons (In this dataset, information about the summer is missing).

- Hr3-24: One-Hot-Encoded hours of the day to a 3 hour interval.

- Traffic_Volume: number of vehicles passing through a road over a period of hour (aggregated average for 3 hours).

- StreetWidth: standardized street width.

- UnitsRes_Prop:  percentage of residential units.

- NumFloors: numbers of floors.

- SubwayProximity: proximity to the Subway (1/NEAR_DIST).

- BoroCode1-5: One-Hot-Encoded of the five boroughs of New York City (Bronx, Brooklyn, Manhattan, Queens and Staten Island).

- LandUse_t1-5: environmental context (Residential, Commercial, Infrastructure, Public Institution, Open Space)

### Exploratory Analysis


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [ ]:
#Load and check data
df = pd.read_csv("../input/nyc-traffic-data/nyc_traffic_ml_orig.csv")
df.head()

In [ ]:
df.info()

In [ ]:
#check for null and missing values
df.isnull().sum().max()

In [ ]:
#descriptive statistics summary
df['Traffic_Volume'].describe()

In [ ]:
#distribution plot
sns.distplot(df['Traffic_Volume'])

A distribution plot of the traffic volume shows the large number of samples belonging to a low traffic volume, and that there are too few examples for higher traffic volume.

I will deal with the imbalance of our target values later.

In [ ]:
#the correlation of the target variable
df = df[['Traffic_Volume']+[c for c in df.columns if c != 'Traffic_Volume']]
corrmat = df.corr()
corr_df = pd.DataFrame(corrmat.values[0,:], index=df.columns, columns=['target_corr'])
corr_df = corr_df.drop(['Traffic_Volume'])
corr_df

Since there are so many variables, I will take the variables with highst correlations with our target.

In [ ]:
#choosing the highs correlation variables
k = 15
cols = corrmat.nlargest(k, 'Traffic_Volume')['Traffic_Volume'].index
cols = [col for col in cols if col != 'Traffic_Volume']
cols

In [ ]:
#correlation matrix
corrmat_sel = df[['Traffic_Volume'] + list(cols)].corr()
f, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrmat_sel, cbar=True, annot=True, square=True);

It can be seen that there are high correlations between the daytime hours and the Daylight feature (remember that Daylight is derived from them), so I will remove the daytime hours from the list.

In [ ]:
cols = [col for col in cols if col not in ['Hr9','Hr12','Hr15','Hr18','Hr21']]
cols

now lets categorize the target Traffic_Volume:

In [ ]:
#categorizing the target
max_value = df['Traffic_Volume'].max()
split_1, split_2 = max_value/3, max_value*2/3
print(split_1, split_2)
df['Traffic_Volume'] = pd.cut(df['Traffic_Volume'], bins=[0,split_1,split_2,np.inf], include_lowest=True, labels=[0, 1, 2])

In [ ]:
# summarize distribution
y = df['Traffic_Volume'].values
counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
plt.bar(counter.keys(), counter.values())
plt.show()

It can be seen that there is a problem of highly imbalanced classes, which can make our classifier to be more biased towards the majority class, causing bad classification of the minority class.
To deal with this problem I will use SMOTE Oversampling which synthesizing new examples of the minority classes so that the number of examples in the minority class matches the number of examples in the majority classes.

### Modeling

In [ ]:
#train test splitting
X = df[cols]
y = df['Traffic_Volume']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

In [ ]:
# transform the dataset
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

plt.bar(counter.keys(), counter.values())
plt.show()


In [ ]:
#Building a random forest classifier
clf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
#evaluate a model
def evaluate_model(X, y, model):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

scores = evaluate_model(X_train, y_train, clf)
scores.mean()

I got an accuracy that looks pretty good, but keep in mind that for imbalanced data the accuracy metric is not a good metric for measuring the correctness of our model.

To get a better idea of how our model is performing I will use Confusion Matrix.

In [ ]:
#Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_pred)
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n" for v1, v2 in zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(3,3)
ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
ax.set_title('Confusion Matrix\n');
ax.set_xlabel('\nPredicted Traffic Category')
ax.set_ylabel('Actual Traffic Category ');
ax.xaxis.set_ticklabels(['0','1', '2'])
ax.yaxis.set_ticklabels(['0','1', '2'])
plt.show()

From the Confusion Matrix it can be seen that the results are not very good, and the model tends to classify the classes with the higher traffic to the lower class. But given the large imbalance in the given data (94.7% - 5% - 0.3%), the model still classifies most of the samples in each class correctly.

Probably if we have more samples with high traffic volume (not just 77), our model will give better results and can provide us more reliable predictions.

